In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
from scipy import stats

In [2]:
def summarize_widths(filtered_df, width_column, output_csv='width_summary.csv'):
    """
    Summarizes the widths for a given filtered DataFrame (either inner/outer membranes or lumen).
    
    Parameters:
    - filtered_df (pd.DataFrame): The DataFrame filtered to either inner/outer membranes or lumen.
    - width_column (str): The name of the column containing the widths to be summarized.
    - output_csv (str): The path to the output CSV file.
    
    Returns:
    - summary_df (pd.DataFrame): The summary DataFrame containing mean widths and standard deviations.
    """
    # Initialize an empty list to store the summary data
    summary_data = []

    # For each process, collect the width for each strip and the overall mean width
    for process in filtered_df['process'].unique():
        process_df = filtered_df[filtered_df['process'] == process]
        
        # Initialize a list to store the mean widths for each strip
        strip_mean_widths = []
        strip_width_sd = []
        
        for strip in process_df['strip'].unique():
            strip_df = process_df[process_df['strip'] == strip]
            mean_width = strip_df[width_column].mean()
            width_sd = strip_df[width_column].std()
            strip_mean_widths.append(mean_width)
            strip_width_sd.append(width_sd)
            summary_data.append([process, strip, mean_width, width_sd])
        
        # Calculate the overall mean width for the process
        overall_mean_width = np.mean(strip_mean_widths)
        overall_sd = np.mean(strip_width_sd)
        summary_data.append([process, 'overall', overall_mean_width, overall_sd])

    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data, columns=['process', 'strip', 'mean width (nm)', 'sd'])

    # Round down to 2 significant figures
    summary_df['mean width (nm)'] = summary_df['mean width (nm)'].round(2)
    summary_df['sd'] = summary_df['sd'].round(2)

    # Save the DataFrame to a CSV file
    summary_df.to_csv(output_csv, index=False)

    return summary_df

def perform_stat_tests(
    filtered_df,
    width_column,
    target_values=(4.7, None),
    confidence_level=0.95,
    alpha=0.05,
    output_csv='stat_test_summary.csv'
):
    """
    Performs one-sample t-tests or Z-tests comparing the mean widths to a target mean for
    a given filtered DataFrame. Calculates confidence intervals and adds a significance 
    indicator.
    
    We don't have a good estimate of the standard deviation of the population, so we will
    use the sample standard deviation as an estimate. This will make the confidence intervals
    wider than they need to be, but it's a conservative approach.
    
    Unless we have a very good reason to believe that the population standard deviation is
    close to the sample standard deviation, we should use the t-test.

    Parameters:
    - filtered_df (pd.DataFrame): The DataFrame filtered to the relevant data.
    - width_column (str): The name of the column containing the widths to be tested.
    - target_values (tuple): A tuple containing the target mean and target standard deviation (if known).
      If target standard deviation is None, a t-test is performed. If provided, a Z-test is performed.
    - alpha (float): Significance level for the tests (default is 0.05).
    - output_csv (str): The path to the output CSV file.

    Returns:
    - summary_df (pd.DataFrame): The summary DataFrame containing test statistics, p-values, confidence intervals, and significance indicator.
    """
# Unpack target values
    target_mean, target_sd = target_values

    # Initialize an empty list to store the summary data
    summary_data = []

    # For each process, perform the statistical test for each strip and the overall data
    for process in filtered_df['process'].unique():
        process_df = filtered_df[filtered_df['process'] == process]

        # Initialize lists to store results for the overall process
        overall_widths = []

        # For each strip within the process
        for strip in process_df['strip'].unique():
            strip_df = process_df[process_df['strip'] == strip]
            widths = strip_df[width_column].dropna()
            n = len(widths)
            if n > 1:
                sample_mean = widths.mean()
                sample_sd = widths.std(ddof=1)
                standard_error = sample_sd / np.sqrt(n)

                # Calculate confidence intervals
                if target_sd is not None:
                    # Z-test
                    z_critical = stats.norm.ppf(1 - (1 - confidence_level) / 2)
                    margin_of_error = z_critical * (target_sd / np.sqrt(n))
                    ci_lower = sample_mean - margin_of_error
                    ci_upper = sample_mean + margin_of_error

                    # Perform one-sample Z-test
                    z_statistic = (sample_mean - target_mean) / (target_sd / np.sqrt(n))
                    p_value = 2 * (1 - stats.norm.cdf(abs(z_statistic)))
                    test_stat = z_statistic
                    test_type = 'Z-test'
                else:
                    # t-test
                    t_critical = stats.t.ppf(1 - (1 - confidence_level) / 2, df=n - 1)
                    margin_of_error = t_critical * standard_error
                    ci_lower = sample_mean - margin_of_error
                    ci_upper = sample_mean + margin_of_error

                    # Perform one-sample t-test
                    t_statistic = (sample_mean - target_mean) / standard_error
                    p_value = 2 * stats.t.sf(abs(t_statistic), df=n - 1)
                    test_stat = t_statistic
                    test_type = 't-test'

                # Determine significance level
                if p_value <= 0.001:
                    significance = '***'
                elif p_value <= 0.01:
                    significance = '**'
                elif p_value <= 0.05:
                    significance = '*'
                else:
                    significance = 'ns'  # Not significant

                # Append the results
                summary_data.append([
                    process, strip, n, sample_mean, sample_sd,
                    ci_lower, ci_upper,
                    test_stat, p_value, significance, test_type
                ])
            else:
                # Not enough data to perform the test
                summary_data.append([
                    process, strip, n, np.nan, np.nan,
                    np.nan, np.nan,
                    np.nan, np.nan, 'N/A', 'N/A'
                ])

            # Collect widths for overall process test
            overall_widths.extend(widths)

        # Perform test for overall process data
        overall_widths = np.array(overall_widths)
        n_overall = len(overall_widths)
        if n_overall > 1:
            overall_mean = overall_widths.mean()
            overall_sd = overall_widths.std(ddof=1)
            standard_error_overall = overall_sd / np.sqrt(n_overall)

            # Calculate confidence intervals
            if target_sd is not None:
                # Z-test
                z_critical = stats.norm.ppf(1 - (1 - confidence_level) / 2)
                margin_of_error = z_critical * (target_sd / np.sqrt(n_overall))
                ci_lower = overall_mean - margin_of_error
                ci_upper = overall_mean + margin_of_error

                # Perform one-sample Z-test
                z_statistic_overall = (overall_mean - target_mean) / (target_sd / np.sqrt(n_overall))
                p_value_overall = 2 * (1 - stats.norm.cdf(abs(z_statistic_overall)))
                test_stat_overall = z_statistic_overall
                test_type_overall = 'Z-test'
            else:
                # t-test
                t_critical = stats.t.ppf(1 - (1 - confidence_level) / 2, df=n_overall - 1)
                margin_of_error = t_critical * standard_error_overall
                ci_lower = overall_mean - margin_of_error
                ci_upper = overall_mean + margin_of_error

                # Perform one-sample t-test
                t_statistic_overall = (overall_mean - target_mean) / standard_error_overall
                p_value_overall = 2 * stats.t.sf(abs(t_statistic_overall), df=n_overall - 1)
                test_stat_overall = t_statistic_overall
                test_type_overall = 't-test'

            # Determine significance level
            if p_value_overall <= 0.001:
                significance_overall = '***'
            elif p_value_overall <= 0.01:
                significance_overall = '**'
            elif p_value_overall <= 0.05:
                significance_overall = '*'
            else:
                significance_overall = 'ns'  # Not significant

            summary_data.append([
                process, 'overall', n_overall, overall_mean, overall_sd,
                ci_lower, ci_upper,
                test_stat_overall, p_value_overall, significance_overall, test_type_overall
            ])
        else:
            summary_data.append([
                process, 'overall', n_overall, np.nan, np.nan,
                np.nan, np.nan,
                np.nan, np.nan, 'N/A', 'N/A'
            ])

    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data, columns=[
        'process', 'strip', 'n', 'mean width (nm)', 'sd',
        'CI Lower', 'CI Upper',
        'test statistic', 'p-value', 'significance', 'test type'
    ])

    # Round numerical columns to 4 decimal places
    numerical_cols = [
        'mean width (nm)', 'sd', 'CI Lower', 'CI Upper',
        'test statistic', 'p-value'
    ]
    summary_df[numerical_cols] = summary_df[numerical_cols].round(4)

    # Save the DataFrame to a CSV file
    summary_df.to_csv(output_csv, index=False)

    return summary_df

# Load the data 
This is already only the dark-adapted data, without the light-adapted strips.


In [3]:
trial_number = 2
run_number = 3

# load the lumen datafile, already filtered to dark-adapted strips
lumen_data = pd.read_csv(f"./output/trial_{trial_number}/csv/lumen_{run_number}_dark.csv")
membrane_data = pd.read_csv(f"./output/trial_{trial_number}/csv/membrane_{run_number}_dark.csv")

# print the lengths:
print(f"lumen: {len(lumen_data)}")
print(f"membrane: {len(membrane_data)}")

lumen: 312
membrane: 360


# Summarize the lumen widths

Test with a t-test to see if the mean lumen width is different from the literature value of 4.7nm. Are we in the range of the observed values from literature? 

Mean lumen width: 4.7, +/- 0.8nm (I interpret this to mean that one standard deviation is 0.8nm)

Ho: The mean lumen width of the dark-adapted strips is 4.7

H1: The mean lumen width of the dark-adapted strips is not 4.7

In [4]:
lit_mean = 4.7 # from the literature, 4.7 nm
lit_sd = 0.8 # from the literature, +/- 0.8 nm. Is this the standard deviation of the population?

# Perform the statistical tests
summary_df = perform_stat_tests(
    filtered_df=lumen_data,
    width_column='lumen_width_nm',
    target_values=(lit_mean, None),
    confidence_level=0.95,
    output_csv='lumen_stat_test_summary.csv'
)

pd.set_option('display.width', 1000)

# filter only to strip == 'overall'
summary_df = summary_df[summary_df['strip'] == 'overall']

# save to disk 
summary_df.to_csv(f"./output/trial_{trial_number}/lumen_stat_test_summary.csv", index=False)

print("Showing overall lumen statistical test summary:")
print(summary_df)

Showing overall lumen statistical test summary:
         process    strip   n  mean width (nm)      sd  CI Lower  CI Upper  test statistic  p-value significance test type
8   0_otsuOffset  overall  52           4.8588  0.8549    4.6208    5.0968          1.3397   0.1863           ns    t-test
17  1_otsuOffset  overall  52           4.7328  0.8374    4.4996    4.9659          0.2822   0.7790           ns    t-test
26  2_otsuOffset  overall  52           4.8588  0.8549    4.6208    5.0968          1.3397   0.1863           ns    t-test
35  3_otsuOffset  overall  52           4.7328  0.8374    4.4996    4.9659          0.2822   0.7790           ns    t-test
44  4_otsuOffset  overall  52           4.8588  0.8549    4.6208    5.0968          1.3397   0.1863           ns    t-test
53  5_otsuOffset  overall  52           4.7328  0.8374    4.4996    4.9659          0.2822   0.7790           ns    t-test


# Inner membrane widths

Test with a t-test to see if the mean inner membrane width is different from the literature value of 11.2 - 11.7nm. Are we in the range of the observed values from literature? 

Mean lumen width: mean of (11.2, 11.7). 
No known sd for the inner membrane width.

Ho: The mean width of the dark-adapted strips is 11.45

H1: The mean width of the dark-adapted strips is not 11.45

In [5]:
# isolate the inner membrane data
inner_membrane_df = membrane_data[membrane_data['membrane_type'] == 'inner']

# expected values for the membrane width
#11.2 to 11.7 nm
lit_mean = np.mean([11.2, 11.7])
print(f"Expected mean: {lit_mean}")

# Perform the statistical tests
summary_df = perform_stat_tests(
    filtered_df=inner_membrane_df,
    width_column='membrane_width_nm',
    target_values=(lit_mean, None),
    confidence_level=0.95,
    output_csv='inner_membrane_stat_test_summary.csv'
)

pd.set_option('display.width', 1000)

# filter only to strip == 'overall'
summary_df = summary_df[summary_df['strip'] == 'overall']
print("Showing overall inner membrane statistical test summary:")
print(summary_df)

Expected mean: 11.45
Showing overall inner membrane statistical test summary:
         process    strip   n  mean width (nm)      sd  CI Lower  CI Upper  test statistic  p-value significance test type
8   0_otsuOffset  overall  44          11.3328  0.7214   11.1134   11.5521         -1.0780   0.2871           ns    t-test
17  1_otsuOffset  overall  44          11.4823  0.6988   11.2699   11.6948          0.3069   0.7604           ns    t-test
26  2_otsuOffset  overall  44          11.3328  0.7214   11.1134   11.5521         -1.0780   0.2871           ns    t-test
35  3_otsuOffset  overall  44          11.4823  0.6988   11.2699   11.6948          0.3069   0.7604           ns    t-test
44  4_otsuOffset  overall  44          11.3328  0.7214   11.1134   11.5521         -1.0780   0.2871           ns    t-test
53  5_otsuOffset  overall  44          11.4823  0.6988   11.2699   11.6948          0.3069   0.7604           ns    t-test


# outer membrane widths

We don't have a number for the expected outer membrane width, so lets just show the summary.


In [6]:
outer_membrane_df = membrane_data[membrane_data['membrane_type'] == 'outer']

outer_membrane_summary_df = summarize_widths(outer_membrane_df, 'membrane_width_nm', output_csv=f"./output/trial_{trial_number}/outer_membrane_summary.csv")

print("Showing outer membrane summary:")
outer_membrane_summary_df = outer_membrane_summary_df[outer_membrane_summary_df['strip'] == 'overall']
print(outer_membrane_summary_df)

Showing outer membrane summary:
         process    strip  mean width (nm)    sd
8   0_otsuOffset  overall             7.20  0.53
17  1_otsuOffset  overall             7.25  0.52
26  2_otsuOffset  overall             7.20  0.53
35  3_otsuOffset  overall             7.25  0.52
44  4_otsuOffset  overall             7.20  0.53
53  5_otsuOffset  overall             7.25  0.52
